In [2]:
from likelihood import *
from detection import *
from Profile_and_Contours import *
import matplotlib.pyplot as plt
import numpy as np
import time
from iminuit import Minuit
from scipy.stats import chi2   
import os.path
import pandas as pd 
import itertools
from flavor_conversion import *
plt.style.use('style.mplstyle')

In [5]:
#Parameters allowed space
#Cooling
Tci, Tcf = 1, 10
tau_ci, tau_cf = 1, 40
Rci, Rcf = 1, 100
tau_i, tau_f = 1, 1.4

#Off-set times
toff_K_i,toff_K_f =0,6
toff_I_i,toff_I_f =0,6
toff_B_i,toff_B_f =0,6

#nubar_e survival probability 
Pee_i,Pee_f=0,1

In [6]:
#Pee
Likelihood_c_Pee.errordef = Minuit.LEAST_SQUARES
x0_c_Pee=[3.6,40,31,1.2,0,0,0,0.5]
m_c_Pee = Minuit(Likelihood_c_Pee, x0_c_Pee)
m_c_Pee.limits = [(Tci, Tcf), (tau_ci, tau_cf), (Rci, Rcf),(tau_i, tau_f),(toff_K_i,toff_K_f),(toff_I_i,toff_I_f),(toff_B_i,toff_B_f),(Pee_i, Pee_f)]
m_c_Pee.fixed["x3"] = True

In [8]:
#Pee
start_time = time.time()
m_c_Pee.migrad()
print("Minuit Pee= %s minutes" % ((time.time() - start_time)/60))
m_c_Pee_BF=np.array(m_c_Pee.values)
m_c_Pee_BF_like=np.array(m_c_Pee.fval)
np.save("Output/BF_Pee_TD_coo.npy", np.array(m_c_Pee_BF,dtype=object))
np.save("Output/BF_like_Pee_TD_coo.npy", np.array(m_c_Pee_BF_like,dtype=object))
m_c_Pee.fixed["x4"] = True
m_c_Pee.fixed["x5"] = True
m_c_Pee.fixed["x6"] = True
print(m_c_Pee)

Minuit Pee= 0.09938782850901286 minutes
┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 244.4                      │              Nfcn = 314              │
│ EDM = 2.01e-05 (Goal: 0.0002)    │            time = 6.0 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │       SOME Parameters at limit       │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────

In [9]:
tau_vec=[1,1.1,1.2,1.3,1.4,1.5]
n_iterate=20
n_points=40
m_c_Pee.fixed["x3"] = True

for tau in tau_vec:
    start_time = time.time()
    
    #Cooling
    m_c_Pee.values=m_c_Pee_BF
    m_c_Pee.values["x3"]=tau
    x_Pee, L_Pee,axu= param_profile(m_c_Pee,"x7",0,1,size=n_points,iterate=n_iterate,subtract_min=False)
    #Saving Arrays
    file_name="Output/Pee_c_tau_"+str(tau)+".npy"
    np.save(file_name, x_Pee)
    file_name="Output/Like_Pee_c_tau_"+str(tau)+".npy"
    np.save(file_name, L_Pee)